In [6]:
from noname import BaseSR, GP

In [ ]:
basic_params = {
    "loss_function": nrmse_loss,
    "n_runs": 10,
    "record_interval": 5,
    "pysr_params": pysr_params
}

In [10]:
a = BaseSR()

In [13]:
b = GP()

In [17]:
from fs_techniques import cmi_feature_selection
from pysr_utils import train_val_test_split

In [4]:
from datasets import load_datasets

dataset_names = [
    "F1",
    # "F2",
    # ("4544_GeographicalOriginalofMusic", "4544_GOM"),
    # "505_tecator",
    # ("Communities and Crime", "CCN"),
    # ("Communities and Crime Unnormalized", "CCUN"),
]

datasets = load_datasets(dataset_names)
width = max([round(len(name)+10, -1) for name in datasets.keys()])

datasets.keys()

dict_keys(['F1'])

In [24]:
X, y = datasets['F1']['X'], datasets['F1']['y']

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y)

a, b = cmi_feature_selection(X_train, y_train, k=5)

In [25]:
print(a)
print(b)

['X2', 'X1', 'X3', 'noise_37', 'noise_26', 'noise_21', 'noise_1', 'noise_41']
[0.39130099727579437, 0.5517024899660466, 0.13435213097056836, 0.044703955892007624, 0.03116691297194104, 0.034514358341909045, 0.03718262267716429, 0.03945842351278695]


In [ ]:
from sympy import lambdify
from shap import SamplingExplainer, utils
from typing import Callable, Optional
import numpy as np
import pandas as pd

def _lambda_func_shap(
    X: np.ndarray, 
    lambda_func: Callable
) -> np.ndarray:
        
    # Unpack each column of X as a separate argument to lambda_func
    return lambda_func(*[X[:, i] for i in range(X.shape[1])])

def shap_feature_selection(
    n_top_features: int,
    X_train_list: Optional[tuple],
    gp_best_equations: Optional[list] = None,
) -> tuple[list[str], list[float], tuple]:
    
    # Initialize SHAP value aggregation
    mean_shap_values = {}
    n_equations = len(gp_best_equations)

    # Process each equation to compute SHAP values
    for gp_best_equation, X_train in zip(gp_best_equations, X_train_list):
        # Convert GP equation to sympy format and extract variables
        sympy_expr = gp_best_equation.sympy_format
        expr_variables = sorted(sympy_expr.free_symbols, key=lambda s: str(s)) # TODO: I dont know why I short
        
        # Skip equations with no variables
        if len(expr_variables) >= 1:
            # Create numpy-compatible lambda function from sympy expression
            lambda_func = lambdify(expr_variables, sympy_expr, modules="numpy")
            str_variables = [str(var) for var in expr_variables]
            
            # Sample data for SHAP analysis
            # Background data provides baseline for SHAP explanations
            X_background = utils.sample(X_train[str_variables], 100, random_state=None)
            # Foreground data is what we want to explain
            X_foreground = utils.sample(X_train[str_variables], 1000, random_state=None)

            # Create SHAP explainer for the equation function
            explainer = SamplingExplainer(
                lambda X: _lambda_func_shap(X, lambda_func),
                X_background
            )

            # Compute SHAP values for each feature in the equation
            shap_values = explainer.shap_values(X_foreground, silent=True)
            # Use absolute values and average across samples to get feature importance
            feature_shap_values = np.mean(shap_values, axis=0)
            # feature_shap_values = np.abs(shap_values).mean(axis=0)

            # Aggregate SHAP values across equations (normalize by number of equations)
            for feature_shap_value, var_name in zip(feature_shap_values, str_variables):
                mean_shap_values[var_name] = mean_shap_values.get(var_name, 0) + feature_shap_value/n_equations

    # Select top features by mean SHAP value
    selected_features = sorted(mean_shap_values, key=mean_shap_values.get, reverse=True)[:n_top_features]
    mean_shap_values_selected_features = [mean_shap_values[feature] for feature in selected_features]

    return selected_features, mean_shap_values_selected_features

In [33]:
from joblib import Parallel, delayed, parallel_backend



def outer_task(i):
    # εδώ τρέχει inner Parallel σε threading
    inner = Parallel(n_jobs=-1)(delayed(lambda x: x**i)(j) for j in range(100))
    return sum(inner)

# μόνο ένα επίπεδο loky
results = Parallel(n_jobs=-1)(delayed(outer_task)(i) for i in range(100))

In [34]:
results

[100,
 4950,
 328350,
 24502500,
 1950333330,
 161708332500,
 13790714119050,
 1200583304167500,
 106177773111333330,
 9507499300049998500,
 859924142434241924250,
 78424986251833195837500,
 7202305859288611690311330,
 665368809336778564827364500,
 61783303007943014596103433450,
 5762496209524731912913787587500,
 539568160957599400822608125097330,
 50697165580103009619183419832730500,
 4778151098135516473529372653422766650,
 451582526294552024729297998923435817500,
 42785619347205807958795562237196787371330,
 4062943470191369347210738338827565614896500,
 386614659739941950597622922368810419475443850,
 36858185883425690907601537265832277990402927500,
 3519983150185588900070347232849412977599288653330,
 336696803388383245486524971717126029933397066662500,
 32253487297794936280903388751996815649470024583385050,
 3093904228427319943051855761295154931986007940007317500,
 297158193435301613031084785362995044864984706753180303330,
 28574695970600432983311605663294002342962361390618264828500,


In [ ]:
from itertools import combinations

a = [1, 2, 3]
b = 4
c = combinations(a, b)

for i in c:
    print(i)